# Segmentation 3D — t2f-only (FLAIR) avec cache NPZ
Notebook autonome prêt à exécuter : dataset basé sur **paires (patient, timepoint)**, **cache** pour accélérer, DataLoaders avec **num_workers=2**, et inférence robuste.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Imports
import os, re, json
import numpy as np
import nibabel as nib

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import torch.optim as optim


In [4]:
# Utils: pad & NIfTI loader (signature: (data, affine))
def pad_to_multiple(volume, multiple=8, mode='constant', value=0):
    """Pad 3D/4D array so spatial dims are multiples of `multiple`.
    Returns (padded_volume, pad_widths)."""
    if volume.ndim == 3:
        shape = volume.shape
        pad_widths = []
        for dim in shape:
            r = dim % multiple
            pad_widths.append((0, 0 if r == 0 else multiple - r))
        return np.pad(volume, pad_widths, mode=mode, constant_values=value), pad_widths
    elif volume.ndim == 4:
        shape = volume.shape[1:]
        pad_widths = [(0,0)]
        for dim in shape:
            r = dim % multiple
            pad_widths.append((0, 0 if r == 0 else multiple - r))
        return np.pad(volume, pad_widths, mode=mode, constant_values=value), pad_widths
    else:
        raise ValueError("Volume must be 3D or 4D")

def load_nifti(path):
    img = nib.load(path)
    data = img.get_fdata()
    return data, img.affine


In [5]:
# Recherche robuste des fichiers (t2f_processed & tumorMask)
from typing import List, Optional

SEQ_KEYWORDS  = ["t2f_processed", "t2f", "flair", "t2_flair", "t2fla"]
MASK_KEYWORDS = ["tumorMask", "mask", "seg", "label"]

def is_nifti(fname: str) -> bool:
    low = fname.lower()
    return low.endswith(".nii") or low.endswith(".nii.gz")

def find_first_matching_file(folder: str, keywords: List[str]) -> Optional[str]:
    keys = [k.lower() for k in keywords]
    for fname in sorted(os.listdir(folder)):
        if not is_nifti(fname):
            continue
        low = fname.lower()
        if any(k in low for k in keys):
            return os.path.join(folder, fname)
    return None


In [6]:
# Modèle UNet 3D (simple) — in_ch=1, out_ch=1
class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv3d(in_ch, out_ch, 3, padding=1),
            nn.InstanceNorm3d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv3d(out_ch, out_ch, 3, padding=1),
            nn.InstanceNorm3d(out_ch),
            nn.ReLU(inplace=True),
        )
    def forward(self, x):
        return self.net(x)

class UNet3D(nn.Module):
    def __init__(self, in_ch=1, out_ch=1):
        super().__init__()
        f = 16
        self.down1 = DoubleConv(in_ch, f)
        self.pool1 = nn.MaxPool3d(2)
        self.down2 = DoubleConv(f, f*2)
        self.pool2 = nn.MaxPool3d(2)
        self.down3 = DoubleConv(f*2, f*4)
        self.pool3 = nn.MaxPool3d(2)

        self.bottleneck = DoubleConv(f*4, f*8)

        self.up3 = nn.ConvTranspose3d(f*8, f*4, 2, stride=2)
        self.dec3 = DoubleConv(f*8, f*4)
        self.up2 = nn.ConvTranspose3d(f*4, f*2, 2, stride=2)
        self.dec2 = DoubleConv(f*4, f*2)
        self.up1 = nn.ConvTranspose3d(f*2, f, 2, stride=2)
        self.dec1 = DoubleConv(f*2, f)

        self.out_conv = nn.Conv3d(f, out_ch, 1)

    def forward(self, x):
        c1 = self.down1(x); p1 = self.pool1(c1)
        c2 = self.down2(p1); p2 = self.pool2(c2)
        c3 = self.down3(p2); p3 = self.pool3(c3)

        b = self.bottleneck(p3)

        u3 = self.up3(b); x3 = torch.cat([u3, c3], dim=1); d3 = self.dec3(x3)
        u2 = self.up2(d3); x2 = torch.cat([u2, c2], dim=1); d2 = self.dec2(x2)
        u1 = self.up1(d2); x1 = torch.cat([u1, c1], dim=1); d1 = self.dec1(x1)
        return self.out_conv(d1)


In [7]:
# Dice et loss
def dice_coefficient(pred, target, eps=1e-6):
    # pred et target en float32 (0..1), taille [B,1,D,H,W]
    pred = (pred > 0.5).float()
    target = (target > 0.5).float()
    inter = (pred * target).sum(dim=(2,3,4))
    union = pred.sum(dim=(2,3,4)) + target.sum(dim=(2,3,4))
    dice = (2*inter + eps) / (union + eps)
    return dice.mean()

class DiceLoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.eps = eps
    def forward(self, logits, targets):
        probs = torch.sigmoid(logits)
        inter = (probs * targets).sum(dim=(2,3,4))
        union = probs.sum(dim=(2,3,4)) + targets.sum(dim=(2,3,4))
        dice = (2*inter + self.eps) / (union + self.eps)
        return 1 - dice.mean()


In [8]:
# Dataset t2f-only (paires) + cache NPZ
class MRIDataset3D(Dataset):
    """t2f-only. Paires (patient, tp). Cache optionnel (npz).
    Retourne: (img[1,D,H,W], mask[1,D,H,W], affine, (patient, tp))."""
    def __init__(self, dataset_dir, pairs, cache_dir=None, use_cache=True):
        self.dataset_dir = dataset_dir
        self.pairs = list(pairs)
        self.cache_dir = cache_dir
        self.use_cache = use_cache
        if self.cache_dir and self.use_cache:
            os.makedirs(self.cache_dir, exist_ok=True)

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        patient, tp = self.pairs[idx]
        tp_path = os.path.join(self.dataset_dir, patient, tp)

        cache_path = None
        if self.cache_dir and self.use_cache:
            cache_path = os.path.join(self.cache_dir, f"{patient}__{tp}.npz")

        if cache_path is not None and os.path.isfile(cache_path):
            z = np.load(cache_path)
            vol    = z["vol"]      # (1,D,H,W) float32
            mask   = z["mask"]     # (1,D,H,W) float32 (0/1)
            affine = z["affine"]
        else:
            # t2f
            t2f_path = find_first_matching_file(tp_path, SEQ_KEYWORDS)
            if t2f_path is None:
                raise FileNotFoundError(f"[t2f introuvable] {patient}/{tp}")
            vol, affine = load_nifti(t2f_path)

            # mask
            mask_path = find_first_matching_file(tp_path, MASK_KEYWORDS)
            if mask_path is None:
                raise FileNotFoundError(f"[mask introuvable] {patient}/{tp}")
            mask, _ = load_nifti(mask_path)

            # prétraitements
            vol = vol.astype(np.float32, copy=False)
            vol = (vol - vol.mean()) / (vol.std() + 1e-8)
            vol = vol[None, ...]  # (1,D,H,W)
            mask = (mask > 0).astype(np.float32)[None, ...]  # (1,D,H,W)

            vol, _  = pad_to_multiple(vol, multiple=8)
            mask, _ = pad_to_multiple(mask, multiple=8)

            if cache_path is not None:
                np.savez_compressed(cache_path, vol=vol, mask=mask, affine=affine)

        # 🔒 assure l'affine en np.float64 (4x4) avant retour
        affine = np.asarray(affine, dtype=np.float64)

        return (torch.from_numpy(vol), torch.from_numpy(mask), affine, (patient, tp))


In [9]:
# Entraînement / évaluation
def train_epoch(model, loader, device, optimizer, criterion):
    model.train()
    total_loss = 0.0
    for imgs, masks, _, _ in tqdm(loader, desc="Train"):
        imgs  = imgs.to(device, non_blocking=True)
        masks = masks.to(device, non_blocking=True)
        optimizer.zero_grad()
        logits = model(imgs)
        loss = criterion(logits, masks)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * imgs.size(0)
    return total_loss / max(1, len(loader.dataset))

def eval_epoch(model, loader, device, criterion):
    model.eval()
    total_loss, total_dice, n = 0.0, 0.0, 0
    with torch.no_grad():
        for imgs, masks, _, _ in tqdm(loader, desc="Val"):
            imgs  = imgs.to(device, non_blocking=True)
            masks = masks.to(device, non_blocking=True)
            logits = model(imgs)
            loss = criterion(logits, masks)
            probs = torch.sigmoid(logits)
            dice  = dice_coefficient(probs, masks)
            total_loss += loss.item() * imgs.size(0)
            total_dice += dice.item() * imgs.size(0)
            n += imgs.size(0)
    return total_loss / max(1,n), total_dice / max(1,n)


In [10]:

# Inférence robuste (gère différents formats de metas) + affine (4x4)
def inference_and_save(model, loader, device, out_dir):
    import numpy as np
    import torch, os
    import nibabel as nib
    from tqdm import tqdm

    os.makedirs(out_dir, exist_ok=True)
    model.eval()

    def get_meta_pair(metas, i):
        # ( [patients], [tps] )
        if isinstance(metas, (list, tuple)) and len(metas) == 2 and \
           all(isinstance(x, (list, tuple)) for x in metas):
            return metas[0][i], metas[1][i]
        # [ (patient, tp), ... ]
        if isinstance(metas, (list, tuple)) and len(metas) > 0 and \
           isinstance(metas[0], (list, tuple)) and len(metas[0]) == 2:
            return metas[i][0], metas[i][1]
        # (patient, tp) si B=1
        if isinstance(metas, (list, tuple)) and len(metas) == 2 and \
           not any(isinstance(x, (list, tuple)) for x in metas):
            return metas[0], metas[1]
        # ".../PatientID/Timepoint" ou "...\\PatientID\\Timepoint"
        if isinstance(metas, (list, tuple)) and len(metas) > 0 and isinstance(metas[i], str):
            s = metas[i].replace('\\', '/').split('/')
            if len(s) >= 2:
                return s[-2], s[-1]
        raise ValueError(f"Format metas non supporté: type={type(metas)} exemple={metas}")

    def get_affine_i(affines, i):
        # torch.Tensor
        if isinstance(affines, torch.Tensor):
            a = affines[i] if affines.ndim == 3 else affines
            a = a.detach().cpu().numpy()
        # numpy array
        elif isinstance(affines, np.ndarray):
            a = affines[i] if affines.ndim == 3 else affines
        # list/tuple
        elif isinstance(affines, (list, tuple)):
            if len(affines) == 0:
                raise ValueError("Liste d'affines vide")
            a = affines[i] if len(affines) > 1 else affines[0]
            a = np.asarray(a)
        else:
            a = np.asarray(affines)

        a = np.asarray(a, dtype=np.float64)
        if a.shape != (4, 4):
            raise ValueError(f"Affine devrait être (4,4), reçu {a.shape} (type {type(a)})")
        return a

    with torch.no_grad():
        for batch in tqdm(loader, desc='Inference'):
            imgs, _, affines, metas = batch
            imgs = imgs.to(device, non_blocking=True)
            logits = model(imgs)               # [B,1,D,H,W]
            probs  = torch.sigmoid(logits).cpu().numpy()

            B = probs.shape[0]
            for i in range(B):
                pred = (probs[i, 0] > 0.5).astype(np.uint8)
                patient, tp = get_meta_pair(metas, i)
                affine = get_affine_i(affines, i)

                out_path = os.path.join(out_dir, patient, tp, 'pred_mask.nii.gz')
                os.makedirs(os.path.dirname(out_path), exist_ok=True)
                nib.save(nib.Nifti1Image(pred.astype(np.uint8), affine), out_path)


In [11]:
# Paramètres & DataLoaders (Windows-friendly paths)
# ⚠️ Adapte ces chemins à ta machine. Utilise des slashes '/' ou des raw strings r'...'
DATASET_DIR = r'/content/drive/MyDrive/Data_test'
OUTPUT_DIR  = r'/content/drive/MyDrive/Data_test_segmentation'
BATCH_SIZE  = 1
EPOCHS      = 20
LR          = 1e-4

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'predictions'), exist_ok=True)

# Cache
CACHE_DIR = os.path.join(OUTPUT_DIR, 'cache_npz')
os.makedirs(CACHE_DIR, exist_ok=True)

# Lister patients et timepoints
patients_timepoints = {}
for p in sorted(os.listdir(DATASET_DIR)):
    ppath = os.path.join(DATASET_DIR, p)
    if not os.path.isdir(ppath):
        continue
    tps = sorted([d for d in os.listdir(ppath) if os.path.isdir(os.path.join(ppath, d))])
    if len(tps) >= 3:
        patients_timepoints[p] = tps[:3]   # 2 train + 1 test
    elif len(tps) == 2:
        patients_timepoints[p] = tps       # 1 train + 1 test
    # sinon (1 TP): ignore

print(f"Patients retenus (>=2 TP): {len(patients_timepoints)}")

# Paires (patient,tp)
train_pairs, test_pairs = [], []
for p, tps in patients_timepoints.items():
    if len(tps) >= 3:
        train_pairs.extend([(p, tps[0]), (p, tps[1])])
        test_pairs.append((p, tps[2]))
    elif len(tps) == 2:
        train_pairs.append((p, tps[0]))
        test_pairs.append((p, tps[1]))

print(f"Taille attendue ~ train={len(train_pairs)} | test={len(test_pairs)}")

# Datasets & loaders
train_ds = MRIDataset3D(DATASET_DIR, train_pairs, cache_dir=CACHE_DIR, use_cache=True)
test_ds  = MRIDataset3D(DATASET_DIR, test_pairs,  cache_dir=CACHE_DIR, use_cache=True)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=2, pin_memory=torch.cuda.is_available(),
                          persistent_workers=True)

test_loader  = DataLoader(test_ds,  batch_size=1, shuffle=False,
                          num_workers=2, pin_memory=torch.cuda.is_available(),
                          persistent_workers=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet3D(in_ch=1, out_ch=1).to(device)
optimizer = optim.Adam(model.parameters(), lr=LR)
criterion = DiceLoss()


Patients retenus (>=2 TP): 3
Taille attendue ~ train=6 | test=3


In [12]:
# Entraînement + sauvegarde meilleur modèle
best_dice = 0.0
for epoch in range(1, EPOCHS+1):
    train_loss = train_epoch(model, train_loader, device, optimizer, criterion)
    val_loss, val_dice = eval_epoch(model, test_loader, device, criterion)
    print(f'Epoch {epoch}/{EPOCHS} - Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Dice: {val_dice:.4f}')
    if val_dice > best_dice:
        best_dice = val_dice
        torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, 'best_model.pt'))
        print('Best model saved.')


Val: 100%|██████████| 3/3 [00:03<00:00,  1.21s/it]


Epoch 1/20 - Train Loss: 1.0000 | Val Loss: 1.0000 | Val Dice: 0.0000
Best model saved.


Val: 100%|██████████| 3/3 [00:02<00:00,  1.26it/s]


Epoch 2/20 - Train Loss: 1.0000 | Val Loss: 1.0000 | Val Dice: 0.0000
Best model saved.


Val: 100%|██████████| 3/3 [00:02<00:00,  1.13it/s]


...

Epoch 18/20 - Train Loss: 1.0000 | Val Loss: 1.0000 | Val Dice: 0.0000


Val: 100%|██████████| 3/3 [00:02<00:00,  1.16it/s]


Epoch 19/20 - Train Loss: 1.0000 | Val Loss: 1.0000 | Val Dice: 0.0000


Val: 100%|██████████| 3/3 [00:02<00:00,  1.09it/s]

Epoch 20/20 - Train Loss: 1.0000 | Val Loss: 1.0000 | Val Dice: 0.0000


In [13]:
# Inférence
inference_and_save(model, test_loader, device, os.path.join(OUTPUT_DIR, 'predictions'))
print('Done. Predictions saved to', os.path.join(OUTPUT_DIR, 'predictions'))


Inference: 100%|██████████| 3/3 [00:04<00:00,  1.36s/it]

Done. Predictions saved to /content/drive/MyDrive/Data_test_segmentation/predictions


In [14]:
# ====== Inference-only : charger le modèle entraîné et sauver dans output_masks/ ======
import os
import torch

# 1) Chemins (réutilise OUTPUT_DIR/DATASET_DIR déjà définis plus haut)
CHECKPOINT_PATH = os.path.join(OUTPUT_DIR, "/content/drive/MyDrive/Data_test_segmentation/best_model.pt")
OUT_MASKS_DIR   = os.path.join(OUTPUT_DIR, "output_masks")
os.makedirs(OUT_MASKS_DIR, exist_ok=True)

# 2) Recréer le dataset TEST (patients avec >=2 TP : test = TP3 si dispo, sinon TP2)
patients_timepoints = {}
for p in sorted(os.listdir(DATASET_DIR)):
    ppath = os.path.join(DATASET_DIR, p)
    if not os.path.isdir(ppath):
        continue
    tps = sorted([d for d in os.listdir(ppath) if os.path.isdir(os.path.join(ppath, d))])
    if len(tps) >= 3:
        patients_timepoints[p] = tps[:3]   # (TP1, TP2, TP3)
    elif len(tps) == 2:
        patients_timepoints[p] = tps       # (TP1, TP2)
    # sinon: ignoré

# paires test : 3e si dispo, sinon 2e
test_pairs = []
for p, tps in patients_timepoints.items():
    if len(tps) >= 3:
        test_pairs.append((p, tps[2]))
    elif len(tps) == 2:
        test_pairs.append((p, tps[1]))

print(f"[INFO] Nb paires test: {len(test_pairs)}")

# 3) Dataset/DataLoader test (avec cache si dispo dans ce notebook)
CACHE_DIR = os.path.join(OUTPUT_DIR, "cache_npz")
os.makedirs(CACHE_DIR, exist_ok=True)

test_ds  = MRIDataset3D(DATASET_DIR, test_pairs, cache_dir=CACHE_DIR, use_cache=True)
test_loader  = DataLoader(test_ds, batch_size=1, shuffle=False,
                          num_workers=2, pin_memory=torch.cuda.is_available(),
                          persistent_workers=True)

# 4) Charger le modèle entraîné
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model  = UNet3D(in_ch=1, out_ch=1).to(device)

if not os.path.isfile(CHECKPOINT_PATH):
    raise FileNotFoundError(f"Checkpoint introuvable: {CHECKPOINT_PATH}")

state = torch.load(CHECKPOINT_PATH, map_location=device)
# Compatibilité stricte : si besoin, mets strict=False
model.load_state_dict(state, strict=True)
model.eval()
print(f"[INFO] Modèle chargé depuis: {CHECKPOINT_PATH}")

# 5) Lancer l'inférence et sauver dans output_masks/
inference_and_save(model, test_loader, device, OUT_MASKS_DIR)
print(f"[OK] Masques sauvegardés sous: {OUT_MASKS_DIR}")

[INFO] Nb paires test: 3
[INFO] Modèle chargé depuis: /content/drive/MyDrive/Data_test_segmentation/best_model.pt


Inference: 100%|██████████| 3/3 [00:03<00:00,  1.26s/it]

[OK] Masques sauvegardés sous: /content/drive/MyDrive/Data_test_segmentation/output_masks
